In [7]:
import pandas as pd
import numpy as np
from knockai.log_decode import log_decode


In [8]:
df = log_decode('/Users/danchecketts/dev/learning/Knockai/resources/runs/r02/r2-100k-af.txt')

In [9]:
# Identify the columns in the DataFrame that correspond to the throttle position and air-fuel mixture
throttle_col = 'TPS'
afr_col = 'WB_Oxy'
time_col = 'Secs'

In [15]:
# Remove the other columns
df = df[[time_col, throttle_col, afr_col]]
df.head()

,Secs,TPS,WB_Oxy
0,0.00,20,15.1
1,0.09,20,14.7
2,0.17,20,13.9
3,0.27,20,13.8
4,0.36,20,14.0


In [18]:
# Calculate the derivative of the throttle position and air-fuel mixture
df['throttle_derivative'] = df[throttle_col].diff() / df[time_col].diff()
df['afr_derivative'] = df[afr_col].diff() / df[time_col].diff()
df.head(50)

,Secs,TPS,WB_Oxy,throttle_derivative,afr_derivative
0,0.00,20,15.1,NaN,NaN
1,0.09,20,14.7,0.000000,-4.444444
2,0.17,20,13.9,0.000000,-10.000000
3,0.27,20,13.8,0.000000,-1.000000
4,0.36,20,14.0,0.000000,2.222222
5,0.45,20,14.0,0.000000,0.000000
6,0.55,20,14.2,0.000000,2.000000
7,0.66,20,15.0,0.000000,7.272727
8,0.75,20,15.2,0.000000,2.222222
9,0.84,20,14.8,0.000000,-4.444444


In [20]:
# Find the point where the rate of change in throttle position exceeds a certain threshold
# and when the corresponding rate of change in the air-fuel mixture first exceeds a similar threshold
throttle_threshold = 20
afr_threshold = -5

throttle_start = df[df['throttle_derivative'] > throttle_threshold].index[0]
afr_start = df[df['afr_derivative'] < afr_threshold].index[0]

,Secs,TPS,WB_Oxy,throttle_derivative,afr_derivative
0,0.00,20,15.1,NaN,NaN
1,0.09,20,14.7,0.0,-4.444444
2,0.17,20,13.9,0.0,-10.000000
3,0.27,20,13.8,0.0,-1.000000
4,0.36,20,14.0,0.0,2.222222


In [21]:
# The difference in time between these two points is the time it takes for the throttle being opened to have an effect on the air-fuel mixture
time_to_effect = df.loc[afr_start, time_col] - df.loc[throttle_start, time_col]
print('Time to effect:', time_to_effect, 'seconds')


Time to effect: -2.3000000000000003 seconds


In [ ]:
# 2.39 to 2.47 is throttle open
# 2.75 the AFR raises as the throttle is opened
# Time taken to respond is 0.28 to 0.36 seconds


Now we know the delay, let's regress the AFR against the INJ_PW
We need to exclude any rows where there is accel fuel being added (Accel_ == True)
We need to exclude any rows where the throttle is closed TPS == 20 or snapped open or closed, i.e. TPS derivative > 100
We will also need to estimate the intake velocity as a function of RPM and MAP with a delay and increase.
